In [ ]:
import pandas as pd
df=pd.read_csv('/content/LA_labeled.csv')
df.head()

,sentences,labels
0,Created and shaped by an intoxicating fusion o...,1
1,Aside from amazing natural delights like the G...,1
2,Given the fact that South America boasts overw...,1
3,Nothing could be further from the truth.,1
4,"No matter what the textbooks say, visiting A...",1


In [8]:
import pandas as pd
df_IN=pd.read_csv('/content/India_labeled.csv')
df_IN = df_IN.sample(frac=1).reset_index(drop=True)
df_JKT=pd.read_csv('/content/JKT_labeled.csv')
df_JKT = df_JKT.sample(frac=1).reset_index(drop=True)
df_LA=pd.read_csv('/content/LA_labeled.csv')
df_LA = df_LA.sample(frac=1).reset_index(drop=True)
df_MEA=pd.read_csv('/content/MEA_labeled.csv')
df_MEA = df_MEA.sample(frac=1).reset_index(drop=True)
df_USE=pd.read_csv('/content/USE_labeled.csv')
df_USE = df_USE.sample(frac=1).reset_index(drop=True)

In [9]:
!pip install transformers

In [10]:
!pip install accelerate -U

In [12]:
sentences=list(df_IN['sentences'])+list(df_JKT['sentences'])+list(df_LA['sentences'])+list(df_MEA['sentences'])+list(df_USE['sentences'])
labels=list(df_IN['labels'])+list(df_JKT['labels'])+list(df_LA['labels'])+list(df_MEA['labels'])+list(df_USE['labels'])

In [13]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [14]:

from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model = model.to('cuda')

In [17]:
X = sentences
y111 = list((np.array(labels)+1)/2)
y=[int(x) for x in y111]

In [18]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [19]:
X_train_tokenized.keys()
print(X_train_tokenized['attention_mask'][0])
len(X_train),len(X_val)


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


(27832, 6958)

In [20]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [21]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [22]:
train_dataset[5]

{'input_ids': tensor([  101,  1996,  2350,  2967,  2651,  2024,  1996,  1047, 22494,  1006,
          2013,  1996, 18039,  1998,  7838, 13363,  2555,  1007,  1010,  1996,
          9875,  2078,  1006,  2013,  8581,  9701,  1007,  1010,  1998,  1996,
         16007,  8950,  2063,  1006,  2013,  1996,  4514,  1998,  7102,  1007,
          1025,  2045,  2024,  2116,  2334,  6284, 20576,  2015,  1997,  2122,
          2350,  2967,  1006,  2107,  2004,  1996,  4907,  1010,  2040,  2444,
          1999,  1996, 25770,  4020,  1998,  2024,  8228,  2005,  2037, 22838,
          1010,  2396,  1010,  1998, 15806,  1007,  1012,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [23]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [24]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    num_train_epochs=5,
    per_device_train_batch_size=8

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

Step,Training Loss


In [ ]:
trainer.evaluate()